In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import ast
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tdam.cover import CubicalCover
from tdam.clustering import FailSafeClustering
from tdam.core_old import MapperAlgorithm

from TrafficTDApythonUtils.utils_v3 import *
from TrafficTDApythonUtils.plots import *

In [ ]:
import geopandas as gpd
from shapely import wkt
from utils import read_data

combined_data = read_data()
TM2 = 3826
taiwan = gpd.read_file('../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp')
taiwan = taiwan[(~taiwan['TOWNNAME'].isin(['旗津區', '頭城鎮', '蘭嶼鄉', '綠島鄉', '琉球鄉'])) & 
                (~taiwan['COUNTYNAME'].isin(['金門縣', '連江縣', '澎湖縣']))].to_crs(TM2)
taiwan_cnty = taiwan[['COUNTYNAME','geometry']].dissolve(by='COUNTYNAME')
taiwan_cnty['geometry'] = taiwan_cnty.buffer(0)

# 原始以 0.001 grid 計算出的區域事故及對應索引, 依照 hex_grid 計算出來的GI
grid_gi_df = pd.read_csv('../ComputedData/Grid/grid_gi.csv')
grid_gi_df['accident_indices'] = grid_gi_df['accident_indices'].apply(ast.literal_eval)
grid_gi_df['geometry'] = grid_gi_df['geometry'].apply(wkt.loads)
grid_gi  = gpd.GeoDataFrame(grid_gi_df, geometry='geometry').set_crs(TM2, allow_override=True)
grid_gi['geometry'] = grid_gi.geometry.centroid

county_join = gpd.sjoin(grid_gi[['geometry']], taiwan_cnty, how='left', predicate='within')
grid_gi['COUNTYNAME'] = county_join['COUNTYNAME']

grid_filter = grid_gi[grid_gi['accident_indices'].str.len() > 0]
grid_filter.reset_index(inplace=True)

In [ ]:
# from utils_model import extract_features

# select_group = [
#     # 氣候暫不討論
#     # '天候名稱', '光線名稱',

#     # 道路問題
#     # '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
#     # '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',

#     # 號誌
#     '號誌-號誌種類名稱', '號誌-號誌動作名稱',

#     # 車道劃分
#     '車道劃分設施-分道設施-快車道或一般車道間名稱',
#     '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

#     # 大類別
#     # '肇因研判大類別名稱-主要', '肇因研判大類別名稱-個別', # 聚焦道路類型
#     # '當事者區分-類別-大類別名稱-車種', # 聚焦道路類型
#     # '當事者行動狀態大類別名稱', # 聚焦道路類型
#     '車輛撞擊部位大類別名稱-最初', #'車輛撞擊部位大類別名稱-其他',
#     '事故類型及型態大類別名稱', '車道劃分設施-分向設施大類別名稱',
#     '事故位置大類別名稱', '道路型態大類別名稱',
    
#     # 子類別
#     # '肇因研判子類別名稱-主要', '肇因研判子類別名稱-個別', # 聚焦道路類型
#     # '當事者區分-類別-子類別名稱-車種', # 聚焦道路類型
#     # '當事者行動狀態子類別名稱', # 聚焦道路類型
#     # '車輛撞擊部位子類別名稱-最初', '車輛撞擊部位子類別名稱-其他', # 道路類型很大程度影響撞擊部位，所以不考慮
#     # '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱', 
#     # '事故位置子類別名稱', '道路型態子類別名稱',

#     # 其他
#     # '當事者屬-性-別名稱', '當事者事故發生時年齡', 
#     '速限-第1當事者', '道路類別-第1當事者-名稱',
#     # '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃',

#     # 設施
#     'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',

#     # A1 or A2
#     # 'source',
#     ]

# all_features_list = []

# for rows in range(grid_filter.shape[0]):
#     features = extract_features(grid_filter, combined_data, select_group, rows)
#     all_features_list.append(features)

# all_features_df = pd.concat(all_features_list, ignore_index=True)
# all_features_df.fillna(0, inplace=True)

# all_features_df[['mrt_100m_count_mean', 'youbike_100m_count_mean', 'parkinglot_100m_count_mean', '速限-第1當事者_mean']] =\
#       all_features_df[['mrt_100m_count_mean', 'youbike_100m_count_mean', 'parkinglot_100m_count_mean', '速限-第1當事者_mean']].\
#         apply(lambda x: (x - x.min()) / (x.max() - x.min()))
        
# all_features_df.to_csv("../ComputedData/ForModel/all_features_filtered.csv", index=False)

In [ ]:
# all_features_df = pd.read_csv("../ComputedData/ForModel/all_features.csv")
all_features_df = pd.read_csv("../ComputedData/ForModel/all_features_filtered.csv")

## Start Mapper

In [ ]:
pc = 9

lens = PCA(pc).fit_transform(all_features_df)

pca = PCA(pc).fit(all_features_df)
ratios = pca.explained_variance_ratio_
print(ratios)
print(ratios.sum()) 

In [ ]:
import pickle

In [ ]:
overlaps = [2, 5]
intervals = [7, 8, 9, 10]
detailed_results = []
silhouette_for_intervals = []

for overlap in overlaps:
    for interval in intervals:
        print(f"Processing overlap {overlap}, interval {interval}")
        mapper_algo = MapperAlgorithm(
            cover=CubicalCover(
                n_intervals=interval,
                overlap_frac=overlap / 10
            ),
            clustering=FailSafeClustering(
                KMeans(
                    n_clusters=2,
                    random_state=42
                )
            ),
            n_jobs=14
        )

        mapper_info = mapper_algo.fit_transform(all_features_df.to_numpy(), lens)

        silhouette_for_intervals.append(mapper_info[1])
        result = {
            "overlap": overlap,
            "interval": interval,
            "silhouette": mapper_info[1],
            "mapper_info": mapper_info
        }
        detailed_results.append(result)

        with open(f"../ComputedData/ForMatrixFilter/o{overlap}i{interval}.pkl", 'wb') as file:
            pickle.dump(result, file)

detailed_results_df = pd.DataFrame(detailed_results)

In [ ]:
all_features_df['hotspot'] = grid_filter['hotspot']
all_features_df['hotspot'] = all_features_df['hotspot'].apply(lambda x: 'Hotspot' if 'Hotspot' in str(x) else 'Not Hotspot')
all_features_df['youbike'] = all_features_df['youbike_100m_count_mean'].apply(lambda x: 'Facility' if x>0 else 'No Facility')
all_features_df['hotspot_youbike'] = all_features_df['hotspot'] + '_' + all_features_df['youbike']

for overlap in overlaps:
    for interval in intervals:
        
        detailed_results_df = pickle.load(open(f"../ComputedData/ForMatrixFilter/o{overlap}i{interval}.pkl", "rb"))
        choose = 'hotspot_youbike'
        mapper_plotter = MapperPlotter(detailed_results_df['mapper_info'], 
                                    all_features_df, seed=87, iterations=30, dim=3,
                                        range_lst=[-0.5, 0.5, 0.5, -0.5])

        # def avg_label(data):
        #     return sum(data) / len(data) if len(data) > 0 else 0
        def most_common_encoded_label(data):
            most_common_item = Counter(data).most_common(1)[0][0]
            return most_common_item

        mapper_plot = mapper_plotter.create_mapper_plot(choose, most_common_encoded_label, avg=False)
        full_info = mapper_plotter.extract_data()
        mapper_plotter.map_colors(choose, size=10, threshold=0)
        mapper_plotter.plot(choose, avg=False, set_label=True, size=1000, anchor=1.33,
                            save_path=f"../ComputedData/ForMatrixFilter/Plots/o{overlap}i{interval}.png")